# Swarm Model Regression

This notebook provides a walkthrough of all the code used to perform the experiments in the article "Signatures, Lipschitz-free spaces, and paths of persistence diagrams" by Chad Giusti and Darrick Lee.  

All code for computing path signatures is in `PathSignatures.jl` and all other code is in `swarm_computations.jl`.  

The code is structured into 5 parts, where the output is saved at each step.
1. **Swarm Simulation**: 500 trials of the 3D D'Orsogna model at various $C$ and $\ell$ parameters are simulated.
2. **Persistent Homology**: Persistent homology is computed at every time step for each simulation. Furthermore, we compute the persistence diagrams for the agent-wise subsampled experiments.
3. **Compute Features**: Feature maps (moment map and persistence paths) for the persistence diagrams are computed at every time step.
4. **Compute Kernels**: The kernel matrices between the trials are all precomputed.
5. **Perform Regression**: Regression is performed using support vector regression with the precomputed kernel matrices.

In [1]:
include("swarm_computations.jl")

swarm_regression (generic function with 3 methods)

### 1. Swarm Simulation

The `swarm_simulate_3d()` function generates 500 trials of the 3D D'Orsogna model at $C, \ell \in [0.1, 2]$ chosen uniformly at random. Because we focus on bounded phenotypes, we reject any trial in which any agent moves further than 40.0 units in any direction after $t=200$.  

The parameters we use correspond to those used in *Thermal and athermal three-dimensional swarms of self-propelled particles* (Nguyen et al., Physical Review E, 2012). In particular,
- $\alpha = 1.0$: propulsion strength
- $\beta = 0.5$: drag
- $m = 1$: mass
- $N = 200$: number of agents

Our simulation is run up until $T=400.0$, and discretized using 200 uniformly spaced time points.  

Output: 
- `PP`: position data of each agent at every time step for all trials

Output folder:  `./SW_data/`

In [2]:
swarm_simulate_3d()

### 2. Persistent Homology

We use the `Eirene` package to compute persistent homology at every time step up to dimension 2.  

The argument to the `swarm_compute_persistence()` function is the number of agents to subsample independently and uniformly at each time step.  

Output:
- `B0, B1, B2`: persistence diagrams for each trial and each time point
- `BE`: Betti curve for each trial and time point
- `NBE`: normalized Betti curve (see Section 6.4 in paper) for each trial and time point

Output folder:  `./PD_data/`

In [ ]:
swarm_compute_persistence(200)
swarm_compute_persistence(50)

### 3. Compute Features

We precompute the feature maps (moment map and persistence paths) for all persistence diagrams using the function `swarm_compute_features()`. The argument to this function is the number of agents to subsample, and the corresponding persistence diagrams from the previous step are used.  

The moment map is truncated at level 6, while the path signature of the Betti curves are computed up to level 5. This initial computation is done at higher truncation levels, so that later steps can simply use a subset of these features for lower truncation levels. Normalized features (as in Section 6.4 in the paper) are also computed.  

Output:
- `MO_F, NMO_F`: moment map and normalized moment map features
- `PP_F, NPP_F`: persistence paths and normalized persistence paths features  


Output folder:  `./FT_data/`

In [ ]:
swarm_compute_features(200)
swarm_compute_features(50)

### 4. Compute Kernels

The kernel matrix is precomputed for every experiment to speed up the kernel regression done in the next step. In particular, we compute $K$, a 500 x 500 matrix, where $$K_{i,j} = \langle S(X_i), S(X_j)\rangle,$$ where $X_i$ is the path of features (either moment or perspath) for the $i^{th}$ trial. We also precompute the kernel matrix for the crocker plots, where in this case we have $$K_{i,j} = \langle BE_i, BE_j\rangle,$$ where $BE_i$ is the discretized Betti curves (up to dimension 2) of the $i^{th}$ trial.  

Arguments:
- `ktype`: type of kernel
    - `ktype=1`: signature + moment map
    - `ktype=2`: signature + perspath
    - `ktype=3`: crocker plots
- `ss`: number of agents to subsample
- `st`: number of time points to subsample
- `st_type`: type of temporal subsampling
    - `st_type=0`: no temporal subsampling
    - `st_type=1`: random temporal subsampling
    - `st_type=2`: initial temporal subsampling
- `normalized`: set to `true` to use normalized features
- `lag`: number of lags for the outer signature features
- `in_lvl`: inner truncation level (for moment map or perspath)
- `out_lvl`: outer truncation level (for outer path signature of features)
- `mixed`: compute the kernel matrix for the heterogeneous experiments

Note that `lag`, `in_lvl`, and `out_lvl` are not used for the crocker plot kernel (`ktype=3`)  

Output:
- `K`: 500 x 500 kernel matrix

Output folder:  `./KE_data/`

In [ ]:
## FIRST COLUMN - NO AGENT SUBSAMPLING
ss = 200
normalized = false
mixed = false
in_lvl = 2
out_lvl = 3

for lag = 0:2
    swarm_compute_kernel(1, ss, 200, 0, normalized, lag, in_lvl, out_lvl, mixed)
    swarm_compute_kernel(2, ss, 200, 0, normalized, lag, in_lvl, out_lvl, mixed)
end
swarm_compute_kernel(3, ss, 200, 0, normalized, 0, 0, 0, mixed)

for st_type = 1:2
    for st = [50, 20]
        for lag = 0:2
            swarm_compute_kernel(1, ss, st, st_type, normalized, lag, in_lvl, out_lvl, mixed)
            swarm_compute_kernel(2, ss, st, st_type, normalized, lag, in_lvl, out_lvl, mixed)
        end
        swarm_compute_kernel(3, ss, st, st_type, normalized, 0, 0, 0, mixed)
    end
end

## SECOND COLUMN - RANDOM AGENT SUBSAMPLING (N=50)
ss = 50
normalized = false
mixed = false
in_lvl = 2
out_lvl = 3

for lag = 0:2
    swarm_compute_kernel(1, ss, 200, 0, normalized, lag, in_lvl, out_lvl, mixed)
    swarm_compute_kernel(2, ss, 200, 0, normalized, lag, in_lvl, out_lvl, mixed)
end
swarm_compute_kernel(3, ss, 200, 0, normalized, 0, 0, 0, mixed)

for st_type = 1:2
    for st = [50, 20]
        for lag = 0:2
            swarm_compute_kernel(1, ss, st, st_type, normalized, lag, in_lvl, out_lvl, mixed)
            swarm_compute_kernel(2, ss, st, st_type, normalized, lag, in_lvl, out_lvl, mixed)
        end
        swarm_compute_kernel(3, ss, st, st_type, normalized, 0, 0, 0, mixed)
    end
end


## THIRD COLUMN - RANDOM AGENT SUBSAMPLING (N=50) HETEROGENEOUS TRAIN/TEST
ss = 50
normalized = true
mixed = true
in_lvl = 2
out_lvl = 3

for lag = 0:2
    swarm_compute_kernel(1, ss, 200, 0, normalized, lag, in_lvl, out_lvl, mixed)
    swarm_compute_kernel(2, ss, 200, 0, normalized, lag, in_lvl, out_lvl, mixed)
end
swarm_compute_kernel(3, ss, 200, 0, normalized, 0, 0, 0, mixed)

for st_type = 1:2
    for st = [50, 20]
        for lag = 0:2
            swarm_compute_kernel(1, ss, st, st_type, normalized, lag, in_lvl, out_lvl, mixed)
            swarm_compute_kernel(2, ss, st, st_type, normalized, lag, in_lvl, out_lvl, mixed)
        end
        swarm_compute_kernel(3, ss, st, st_type, normalized, 0, 0, 0, mixed)
    end
end


### 5. Compute Regression

We use kernel support vector regression (SVR) to perform regression against the $C$ and $\ell$ parameters of the model.

#### Setup
We use a 80/20 train/test split, and we perform 100 iterations by shuffling the splits. The same shuffling is used across all experiments.

#### Hyperparameter Selection
There are two hyperparameters for SVR: $C_{SVR}$ and $\epsilon_{SVR}$. These hyperparameters are chosen by a grid search in $C_{SVR} \in [10^{-3}, 10^3]$ and $\epsilon_{SVR} \in [10^{-5}, 10^1]$ on a log scale with 13 points for each parameter. These are chosen by performing 4-fold cross-validatione exclusively on the training data. This hyperparameter selection is repeated for each iteration.  

#### Regression Results
The mean squared error (MSE) for both $C$ and $\ell$ are reported for each iteration.  

Output:
-`K_error`: MSE for all iterations

Output folder:  `./RG_data/`

In [ ]:
## FIRST COLUMN - NO AGENT SUBSAMPLING
ss = 200
normalized = false
mixed = false
in_lvl = 2
out_lvl = 3

for lag = 0:2
    swarm_compute_regression(1, ss, 200, 0, normalized, lag, in_lvl, out_lvl, mixed)
    swarm_compute_regression(2, ss, 200, 0, normalized, lag, in_lvl, out_lvl, mixed)
end
swarm_compute_regression(3, ss, 200, 0, normalized, 0, 0, 0, mixed)

for st_type = 1:2
    for st = [50, 20]
        for lag = 0:2
            swarm_compute_regression(1, ss, st, st_type, normalized, lag, in_lvl, out_lvl, mixed)
            swarm_compute_regression(2, ss, st, st_type, normalized, lag, in_lvl, out_lvl, mixed)
        end
        swarm_compute_regression(3, ss, st, st_type, normalized, 0, 0, 0, mixed)
    end
end

## SECOND COLUMN - RANDOM AGENT SUBSAMPLING (N=50)
ss = 50
normalized = false
mixed = false
in_lvl = 2
out_lvl = 3

for lag = 0:2
    swarm_compute_regression(1, ss, 200, 0, normalized, lag, in_lvl, out_lvl, mixed)
    swarm_compute_regression(2, ss, 200, 0, normalized, lag, in_lvl, out_lvl, mixed)
end
swarm_compute_regression(3, ss, 200, 0, normalized, 0, 0, 0, mixed)

for st_type = 1:2
    for st = [50, 20]
        for lag = 0:2
            swarm_compute_regression(1, ss, st, st_type, normalized, lag, in_lvl, out_lvl, mixed)
            swarm_compute_regression(2, ss, st, st_type, normalized, lag, in_lvl, out_lvl, mixed)
        end
        swarm_compute_regression(3, ss, st, st_type, normalized, 0, 0, 0, mixed)
    end
end


## THIRD COLUMN - RANDOM AGENT SUBSAMPLING (N=50) HETEROGENEOUS TRAIN/TEST
ss = 50
normalized = true
mixed = true
in_lvl = 2
out_lvl = 3

for lag = 0:2
    swarm_compute_regression(1, ss, 200, 0, normalized, lag, in_lvl, out_lvl, mixed)
    swarm_compute_regression(2, ss, 200, 0, normalized, lag, in_lvl, out_lvl, mixed)
end
swarm_compute_regression(3, ss, 200, 0, normalized, 0, 0, 0, mixed)

for st_type = 1:2
    for st = [50, 20]
        for lag = 0:2
            swarm_compute_regression(1, ss, st, st_type, normalized, lag, in_lvl, out_lvl, mixed)
            swarm_compute_regression(2, ss, st, st_type, normalized, lag, in_lvl, out_lvl, mixed)
        end
        swarm_compute_regression(3, ss, st, st_type, normalized, 0, 0, 0, mixed)
    end
end
